In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
from grid import Grid
my_aoc = aoc.AdventOfCode(2018,20)

In [2]:
def move(grid, direction):
    """
    Function to move to the next room in the grid.
    This is a bit more complex than Grid().move() because:
      - we are creating the rooms as we go
      - moving to a room is 2 move (door, room)
      - we are identifying doors as we go through them
    
    Args:
        grid: Grid() object
        direction: str() movement direction 'n', 's', 'e', 'w'
    """
    # map doors by direction
    door = {
        'e': '|',
        'w': '|',
        'n': '-',
        's': '-'
    }
    # get the neighbors of the current position
    neighbors = grid.get_neighbors()
    # save door location so we can mark it after we go through
    door_location = neighbors[direction]
    # move direction twice
    grid.move(direction)
    grid.move(direction)
    # mark path as door after moving through it.
    grid.set_point(door_location, door[direction])
    # get neighbors of new room position
    neighbors = grid.get_neighbors(directions=['ne', 'nw', 'se', 'sw', 'n', 'w', 'e', 's'])
    # init walls if they are not already
    # walk neighbor directions
    for n_dir in ['ne', 'nw', 'se', 'sw']:
        # get neighbor
        neighbor = neighbors[n_dir]
        # get neighbor value
        value = grid.get_point(neighbor, '%')
        # if oob, make in bounds
        # FIXME: this seems like a Grid() issue.  This grid is infinite
        # so there should be no oob
        if value == '%':
            value = '#'
            # set point of wall
            grid.set_point(neighbor, value)
    # walk door direcdtions
    for n_dir in ['n', 'w', 'e', 's']:
        # get neighbor
        neighbor = neighbors[n_dir]
        # get neighbor value
        value = grid.get_point(neighbor, ' ')
        # if default value (door not already set), then mark unkown
        if value in ' ':
            value = '?'
            # set value
            grid.set_point(neighbor, value)
    # set room value
    grid.set_point(grid.pos, '.')
    # update grid data
    # grid.update


def set_start(grid, pos=(1,1)):
    # set start position
    grid.pos = pos
    grid.overrides = {grid.pos: '*'}

def follow_path(grid, path):
    set_start(grid)
    for direction in path:
        move(grid, direction)
    set_start()

In [3]:
input_text = """#?#
?.?
#?#"""
input_lines = [
    "^ENWWW(NEEE|SSE(EE|N))$"
]
my_grid = Grid(input_text, coordinate_system='cartesian', type='infinite')
my_grid.pos = (1,1)
print(my_grid)
follow_path(my_grid, 'nneennwwnnwwss')
print(my_grid)
follow_path(my_grid, 'wwwsssseeeeeeeennneennn')
print(my_grid)

#?#
?*?
#?#


TypeError: set_start() missing 1 required positional argument: 'grid'

In [8]:
from heapq import heappop, heappush

def follow_path2(grid, line, line_ptr=0, start_position=(1,1)):
    set_start(grid, start_position)
    skip_idx = -1
    for idx in range(line_ptr, len(line) - 1):
        if idx < skip_idx:
            continue
        char = line[idx]
        if char in 'news':
            move(grid, char)
        elif char == '|':
            set_start(grid, start_position)
        elif char == '(':
            current_pos = grid.pos
            skip_idx = follow_path2(grid, line, idx + 1 , current_pos)
        elif char == ')':
            return idx + 1
        elif char == '$':
            return
        elif char == '^':
            pass
        else:
            print(f'Unknown character: {char}')
    return

def adjacent_rooms(grid, point):
    neighbors = grid.get_neighbors(point=point)
    rooms = []
    offsets = {
        'n': (0, 2),
        'e': (2, 0),
        's': (0, -2),
        'w': (-2, 0)
    }
    for direction in 'news':
        if grid.get_point(neighbors[direction]) in '|-':
            pos_x, pos_y = point
            rooms.append((pos_x + offsets[direction][0], pos_y + offsets[direction][1]))
    return rooms

def map_rooms(grid):
    current_pos = (1,1)
    set_start(grid, current_pos)
    heap = []
    paths = {}
    max_doors = 0
    heappush(heap, (0, current_pos, ()))
    already_seen = set()
    while heap:
        doors, current_pos, path = heappop(heap)
        if current_pos in already_seen:
            continue
        max_doors = max(max_doors, doors)
        path = list(path)
        path.append(current_pos)
        path = tuple(path)
        already_seen.add(current_pos)
        paths[current_pos] = path
        next_rooms = adjacent_rooms(grid, current_pos)
        for room in next_rooms:
            heappush(heap, (doors + 1, room, path))
    return max_doors

input_text = """#?#
?.?
#?#"""
my_grid = Grid(input_text, coordinate_system='cartesian', type='infinite')
line = "^ENWWW(NEEE|SSE(EE|N))$"
line = "^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$"
line = "^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$"
line = "^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$"

follow_path2(my_grid, line.lower())
my_grid.update()
set_start(my_grid)
map_str = str(my_grid)
map_str = str(my_grid).replace('?','#').replace('*','X').replace(' ','#')
print(map_str)

print(map_rooms(my_grid))

###############
#.|.|.|.#.|.|.#
#-###-###-#-#-#
#.|.#.|.|.#.#.#
#-#########-#-#
#.#.|.|.|.|.#.#
#-#-#########-#
#.#.#.|X#.|.#.#
###-#-###-#-#-#
#.|.#.#.|.#.|.#
#-###-#####-###
#.|.#.|.|.#.#.#
#-#-#####-#-#-#
#.#.|.|.|.#.|.#
###############
31


In [10]:
import re
from heapq import heappop, heappush
def expand_string(line):
    pattern_group = re.compile(r'\(([^()]+)\)')
    # Remove ^ and $ as they're just markers
    line = line.strip('^$')
    heap = []
    final_strings = set()
    heappush(heap,(line.count('('), line))
    while heap:
        count, line = heappop(heap)
        # fully resolved, add to final_strings
        if count == 0:
            final_strings.add(line)
            continue
        match = pattern_group.findall(line)
        if match:
            replace = f"({match[-1]})"
            replacements = match[-1].split('|')
            for replacement in replacements:
                new_line = line.replace(replace, replacement)
                heappush(heap, (new_line.count('('), new_line))
    return final_strings


line = "^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$"
line = "^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$"
expanded_strings = expand_string(line)

my_grid = Grid(input_text, coordinate_system='cartesian', type='infinite')
for path in expanded_strings:
    follow_path(my_grid, path.lower())

map_str = str(my_grid).replace('?','#').replace('*','X')
paths = []
for point in my_grid:
    if my_grid.get_point(point) == '.':
        shortest_paths = my_grid.shortest_paths((1,1), point, invalid=['#','?'])
        paths.append(shortest_paths[0])
most_doors = 0
for path in paths:
    doors = 0
    for point in path:
        if my_grid.get_point(point) in '|-':
            doors += 1
    if doors > most_doors:
        most_doors = doors
print(most_doors)



TypeError: set_start() missing 1 required positional argument: 'grid'

In [7]:
input_text = """^WSESESSENENESSEENNW(NWWNWW(SESNWN|)NENENWWNEENEESESENNEENWNWWS(S|E|WNNWNWSWSWNNENNWNWSWSE(SSSWNWN(WWNNNWSWSESWWSWSWNWSWSEESSEENN(WSNE|)ESSEENWNENWN(WSWENE|)EESESSENEES(WSWWSSS(WNN(NNN|WSSSSWWNENWNN(ESNW|)WSSWWWWSWNNEEEENWWWNENWWWSS(ENSW|)WSSE(N|SSESWWSSWSSWWSESENESSSSWNWWWSWSSSEESWSWWN(NNWNENWNNNESSENEN(ESEEWWNW|)NNWWS(ESNW|)WWSSWSESSSWSWNNENWWNENWNWWNNWWWWSWSSWWWWNENWWSSSESSSSSWSSWWWNENWWSWWSSSSESENESSSSENENWNENESENENENNEENNWSWW(NNNES(EEEESSENEEENE(SESEEE(NW(NEWS|)W|SSSSESSWWN(WWWNWNENWNN(WSSWSESWWNWWN(WWSESSSSSEEENESESWSWNWSSWWWNEN(ESNW|)WNNNNW(NEWS|)SSSSWSSE(SEEEESWSESWSESSENNNNNESESESWW(N|SESSW(N|WSESWSSESWWNWSSEESSWSESWSEENNEENWNEEEESSWW(NEWS|)SWSEEESWWWW(WWWWWWWNNNNWNWSSSWSWS(EEEN(W|NN)|WNWWNENWNEEES(WSEWNE|)ENNNWWWWS(WSSWWNNNE(ENENNNWSWNWSSE(SWWSSSWSSSS(ENEN(WNSE|)EESS(W(W|N)|EE)|WNNWWSS(ENSW|)WNNNNNENNNNW(NEESSEE(SSWS(E|SW(NNNEWSSS|)S(WNSE|)E)|NE(NWW(NNENWWNNWNNNENEEEESWSSWNNWSWSESES(W|ESENEEEESEENNNWNNEEESESEENN(ESSSSWSWWSSWNW(WSSWSSS(ENNEESS(WNSE|)EES(WW|ENNNE(NWNN(EE(S(W|E(SSWNSENN|)N)|N(N|W))|WSSSSWNNWWN(SEESSEWNNWWN|))|SSSSW(W|SEE(NEWS|)SSWW(NE|SEE))))|WNNNNENWWSWNW(NEEWWS|)SWNWSW)|NNNN(NWES|)ESS(EEN(W|E)|S))|W(WNWWNWNWSS(WWSSE(SWS(EESNWW|)WNWSW(W|NNENE(S|NWNENWNENWNN(EES(SSE(SWSEWNEN|)EEEE(NW(NEN(W|E(N|SS))|W)|S(S|W))|W)|WWWSSWSWSWS(EENESENE(NW(NNESNWSS|)W|S)|SW(SSSSSESSS(WNN|EN)|NNNNNEE(SWSNEN|)NNE(EENNE(SSEWNN|)NNWNENEENEENNNNNWNWNWSSESWSWSEES(WSWWN(E|NNNNNNNE(NNWNEEEEESSENESENENWNWNNNEESS(WNSE|)SENENNW(S|NNENNENENNWWWNWWWNNWSSSSSSESEE(SSWNWSWNNWNWWWWNNEENNNNENE(SSSW(SESWS(WW|E)|N)|ENEEENNWNWSWS(EE|WSWWSWSSWSESWWWNWSWSWSW(SSSEENEEE(EEESE(N|ESWSSEE(NWES|)SWWS(EESEWNWW|)WWNENNWN(E|NWWSESWWNW(NEWS|)WWSW(N|SSSESWSSENENNNWNEE(NWES|)SES(ENE(S|N(E|WW))|SW(SSWSWWSESSSENNNESSE(SSSWWSEESS(SSWSWWSS(ENESENNE(WSSWNWESENNE|)|W(SSENSWNN|)NNNNENN(EESWS(SWEN|)E|NNN(EENSWW|)W(NNN|SSSS)))|E)|NNN(W|N(N|E)))|N)))))|NWNW(NEESNWWS|)SW(WSNE|)N)|NNENENENNWNEESENNWWNWSWNNW(SSSESWSS(ENEWSW|)S|NENESEEEENESEEEE(SWSWNWSS(E|WNW(WWN(WW|EEE)|SS(E|SSWS(WNSE|)S)))|NWWWNNWWS(E|WSWWNWWNW(SS|NENWNENNW(S|NENENNWSW(S|NNEENWWNNNNNEEENESENNWWWSWNNEENNWWS(E|W(SSS|NNENENNNWNNENWNNW(NEESSEESENNN(WWSEWNEE|)ESENESSWWSEESSEEENENWNWNW(SSESWENWNN|)NEEEESS(WNWESE|)ENNESSESENEENEN(EESSSEEEESSWWSSENESEENWNENNEESEEENNNWN(WSWNWWSS(W(SS|WWNEN(ESNW|)WWWSSEN(SWNNEEWWSSEN|))|E(EEESWENWWW|)N)|EEEEEEESESENN(W|EESEESSWSWNW(N(EE|N)|SWNWSSWSESWSSSWWSEEESSWWWN(WSWNNNNE(SS|ENWNNWSWWSE(SSWSSE(SSEEESWSWSEENESSWSESESWWSSWWNWNEE(S|N(NNWWNWNWSWWWWSEEESENESEESWWWWSESE(SSE(N|SENESSENNENESEESSESSEESWWWNNWWW(NEN(ESNW|)W|SEESSSSSEESSWSEENENENWNENWN(WSS(SS|WNNWSS)|ENENWNWNEEEESESSSWW(SW(N|SEESEEENENNNWSSW(W(SEWN|)W|NNNNENWWNEEEESENENWWWWWNWSWSWWNENNNWWSWWSEE(ENSW|)SSWW(SS|WN(EE|NWNENEENNNEESES(WWNSEE|)EENNESESWSS(WNWSSNNESE|)EES(W|EEEENESEESWSEEESWSWNWSSWSSWSESWSSSSESESESWWWN(WSSESENESSSES(WWNNWW(SESNWN|)NWNNWS(SSEWNN|)WNWNEEENWWWWWNNWN(EESENENENNNNNN(EEE(SSWSW(NNEWSS|)SSSE(NN|SWWSEES(SEWN|)WWW(WW|N))|N(W|NE(NWES|)SS))|W(SWEN|)N)|WW(NEWS|)SWSWSWWSWNWSSWNWSWNWWSWNNENENEENWNENE(SSSE(NEWS|)SWWWS(W|EE)|NNWSWSWWWNNNWWNWSSWNWWNENNES(EENN(W(NN(ESNW|)NWSSSWNWSSWNWWWSWNWNNWWWSSWSSSSSENNNEN(W|NNESSESESESWSEENENWN(W|E(NWES|)E(E|SSSEESSSWSSWSSENEENESSWWSSSENEN(ENE(NENWW(S|NNNE(SSEE(ENWWEESW|)S|NNWSWWSSS(WS|ENN)))|SSSW(WSWSEEEN(ESENN(E(N|EESESENN(ESENN(W|ENES(SWSSWSW(WWWWSWN(NNESEWNWSS|)WWWSWNWWSESESSEEENWWNEE(NWES|)EES(ENNEWSSW|)WSSWWWWWSESSENEN(W|ESSWSWSEE(NESNWS|)SWWWWSESEE(SWWWNWSWWWS(EE|WNNENESENENNNNWSWNNEE(ESSSEN|NNNENWNWNWNNE(S|NWNWWNEN(ESNW|)WWWSWNWNWNNNWWSSWSWNWSSEEEE(N(NN|W)|ESWWSWNWSSSS(WNNNW(WSS(WNNW|EN)|N(NNNWNENENWNNNEE(SWSESSE(SWWEEN|)NENEN(WWSNEE|)ENWNNE(N|SE(N|SSESEEN(E(SE(NESNWS|)SWS(E|SWWSS(ENESNWSW|)WNWNNE(S|N(WNWSW|EESW)))|N)|W)))|NNWWNEENWWWSSS(EE|WWNWNNWSSSWWSEEE(EEESSWSESWSWNNWNEN(E|WWW(SESWENWN|)WWNNWWNNWNNW(SWSSSSSES(W|E(ENENWW(S|WNN(ESNW|)N)|S))|NENWNENNENWWS(S|WNW(NEEEN(WWW|ESENNNEENENNWSWNWNENWWNEENNWWNNN(EESS(WNSE|)EESSSW(SSENEENNEESEENWNNWN(WSS(E|WW(NEWS|)SS)|NNEESS(WNSE|)SSENNEESSSSWN(WSSWSSWSWNWWW(NEEEN(WN|ES)|SSESEN(NWES|)EEEEN(W|NEN(ESEESSESWSWSEEEEESESWSSESENESES(ENNWNN(EES(S|W)|WSW(W|NNNNE(SS|ENNEEES(SWWNEWSEEN|)ENN(ESSE(E|S)|WNNWWWNNWN(WSSWSWWWSS(EEEN(ENE(N|SS(EEN(ESNW|)W|SSWWN(ENSW|)WSS(ESNW|)WNW(NE|SW)))|WW)|WNNWWN(WW|EEENWNNWSW(SEWN|)NNEENESSSE(NNNN(EE|WWWSWNN(WSWWEENE|)EEN(W|N))|SE(SW|ENW))))|EENESSWSEEE(N(WNENWNN(WW(SEWN|)N|EEE(SWWEEN|)NNNEN(W|ESSSEE(NWNNESESS(NNWNWSNESESS|)|SSW(NWWNN|SESS))))|E)|SS))))))|WSWWN(E|WW(SEWN|)N(E|WWNNENWWSWSESWWSSWNNNNNNNE(NWWN(EENWNEESS(NNWWSEWNEESS|)|WWSW(SEE(N|ESWWWWSSSEENN(WSNE|)EESSW(N|SSE(N|SS(WWN(E|WWS(SSE(NN|S)|WNWNNWSWWNNE(NEEE(NWNENN(ESNW|)WW(NEWS|)S(WW(NEWS|)SS(ENESNWSW|)SW(SWSSSE(SS(WNSE|)ENESSWWSEEENNN(NWWEES|)E|NN)|N)|E)|S(SES(EENWESWW|)W|WW))|S)))|E))))|N))|SS(SS|ENEEEESSSW(ENNNWWEESSSW|))))))|W)))|NN))|NN)|WW(N|S(SE(SWSE(EE|SS(SSE(S(ESEWNW|)W|N)|W(N|W)))|N)|W))))|S))))|N)))|E))|EEEESWWSESWSESWSWN(N|WSS(W|EESWSESWSW(SW(N|SESWW(NN|WS(EESE(SWEN|)NENEEENWWN(WSNE|)NESEENNESENNNWNENESSESE(SWS(EEENWESWWW|)SW(NNN|SWS(WWNENE|ESS(WN|ESEN)))|NNNNNNN(WNNWSWSSSEN(N|ESS(SS|WWWSWW(SESS(E|WNW(SSESNWNN|)NN)|NENENWW(NENE(S|NNE(S|NWN(EE(SEES(SS|WW)|N)|W(N|WWSWS(WNSE|)EEE(NWES|)S))))|S))))|ESESSSWNN(SSENNNSSSWNN|)))|WWSSW(SESWSSEE(NWES|)SWS(WNSE|)(E|SS)|WNENWW(S|WNENWWSS(NNEESWENWWSS|))))))|NNN))))))))|N(E|W)))|N)|EENESEE(SWWSSEN(SWNNEEWWSSEN|)|N(ESNW|)WNNW(N|WS(WSNE|)E))))|W))|W)|W)|N))|W))))|S)|ESESWSEEEN(WNSE|)ESS(SWNWSSE(WNNESEWNWSSE|)|EE))|S)))|SESEEENWWNWNEENNWWN(WSSEEWWNNE|)EENNWNW(SSEWNN|)NEEENENWNWSSWWNWWS(WNNENENESE(NESENNEEENWWWNNW(WSESWS(E|WNW(NEWS|)SW(S|N))|NENWNEENNWWWNNWNNNWWWSWNWSSSEEN(EE(NWES|)SSS(E(SE(SWEN|)EE|N)|WN(N|WWWSWNW(W(W|NEENNWSWNWNNESENE(S|ENENWNWSSWNNWWWSWNWSWSSENEE(ENESNWSW|)SSW(WW(SESNWN|)WNNNNWSWSWSS(WNNWN(WSW(NWSNES|)SEESWWSSEEN(ESS(ENNSSW|)WWS(E|S(WW|S))|W)|EENENWNNNNWSWS(ESWSSEN(SWNNENSWSSEN|)|WNNENNEN(ESS(EENENENENNESESWSEEEENENNNN(WWSWW(SESSENEN(N|W)|WWN(WSW(NWWEES|)SESWSWSWS(NENENEWSWSWS|)|EEE))|ESSSSSSEENEENNWWS(W(S|NNNNESES(W|ENN(W|ESENESSWSSENESESWWWSW(SEEEEN(WW|ESESWSEESENESSSSSESSWNWNNWWWNNWWWNENN(WSWWWN(NW(NEWS|)WSWWN(WN(WSWSWSSESENEESS(W(N|WWWWWNNE(SEWN|)NNWNN(ESENEWSWNW|)W(N|WSESS(WSWWW(NEN(E(S|N|E)|W)|SESENEE(SWSNEN|)N)|E)))|ESESEESS(ENNNEEESSEEEN(ESSESSSSEESEESWWWSWSESWSSESESWSWW(SESWSWSSW(NNWNEN(ESNW|)W|SWSW(SSEESWSW(SSS(W|EEENNENENWNNESESESWSEENENNW(NENWNNEES(W|EESESWSEESSSWNWN(WNNW(NEWS|)SSSESWWSESSWSWWWWSSSENNESESENN(W|ENENNNE(N|EESSSSWNNW(NEWS|)SSSSESSESSEEEEEEESEEESESWWSSSSEEESWSESEEESEENWNWWNW(NEEENNENWWWWNNNNWS(WNWWWNEEEEEESSSEEES(WWW|EENNNWWNEENWNWWNWNENESENNWWNNWWWWSSSWNNWNENEEEEEESEES(WW|ESSEE(NWNNWNWNW(NEEENE(NWNENNNWWSESWWWWSEESS(ENNSSW|)WNWWWWSEEESWWS(EE|WNWNNWNWSWWNENNENNWNEESENEENWWWWWNEENWWNWSSSSWSWNWWSWSWWWSW(SSEEN(W|EEEN(WWW|NE(SESWSSESWSWNWSSEEENESSWSSWWNENWWWSWSW(SEENE(SSWSWW(NEWS|)SW(NWES|)SESSENNNEEEN(W|ENN(WSNE|)EEE(ESSSWNNWWSSE(SWW(SWWS(WNNEEWWSSE|)EEENEEEN(W|EEES(EE(E(E|SSW(SEEESNWWWN|)N)|NWNWWWNEENNNESE(SWSNEN|)N)|WW))|N)|N)|NWNENENNEE(S(E|W)|NWWW(WNN(W|NNN(W|E(E|SS|N)))|SS))))|N)|NNNE(S|ENWWNNESENEN(ESNW|)W))|N)))|NW(NWSWNNEENWWWSSWNNNNW(NEEESWSEENNENNWWNWSWS(EEE|WNWN(NNNESSSENNNESENEESEEEEEENNNESENNEEESSWW(NEWS|)SWWSESSWWWWSESWWSEEESWSWSWNWW(N(WSNE|)E(NNW(S|NNWN(EEE(SSWNSENN|)EEEE|W(S|N|WW)))|EE)|SSE(N|ES(WW|ENENENESENE(S|NWWNEENN(WWS(WNSE|)E|NENEEESWWSWSEESENESSSEENWNNNENNWNENWWWNENWW(SSWWS(WSNE|)EEEESS(SWWEEN|)E|NENNNNENWNWSSWWNNE(NEEESEEESENNWWWNEEEESSSESEENWNNW(S|NEESSE(SSSWSESSSSSSWSWSWWNWW(SSSEN(N|ESENEENE(N|SSWSW(SESWSWSS(WSESWWNWWSSS(ENNSSW|)WWNENN(NEEENWNENWNN(WNSE|)ESEE(NW|SWSS)|WWS(SWEN|)E)|ENESENN(NNN|W))|N)))|NNNWNNEES(ENNWWNNN(WNWSSESSWW(SESSNNWN|)N(N|E)|EESWSEENESSESWW(N|SEESWS(E|WWN(WSSEE(E|S)|E))))|W))|NNNNNNNNWNENNWWWSWSEE(NEWS|)SWWWWSESESENN(W|EESSSW(WWS(EEE|WWWNNWWSS(ENSW|)SWWNNNWWWNNNWSSSSEESWWSWWNENWNWSSWSESSENEESWSWWSESWSWSWWSESESS(ENNN(EN(W|NENEN(NE(SSESENESSWWWW(WS|NE)|NNEN(ENNSSW|)W)|W))|W)|WWWNWNWWW(SEEWWN|)NWWNWNEN(WWSSNNEE|)NNENWNEEESSEEENENWWW(SEWN|)NEEEE(SSSSW(N|WWSWSWS(ES(W|ENNE(SSSESE|EEN(ENNNNSSSSW|)WW))|WWNNE(EN(E|W(W|NN))|S)))|NWNNNNNEEENWWNEEENNWNWSWNWSWNN(EEEEESESSSES(WWSESS(S|WNWSWW(SESW|NE))|ENNNENESSSSSSS(EE(NEEE(S(ESSENSWNNW|)WW|NWWNNNESSENEE(SWEN|)NWWNNESENN(WWWSWNWSS(SW(SESSW(N|S)|NNNWWWW(S(E|SS)|W))|EE)|ESEEN(EESWS(WWWSNEEE|)E|W)))|SSS)|WNNNNN))|WSWSESWWWWWNWWSSSEN(EESWSWWWSS(ENEEEEESSWWSESE(SWWWW(S|NNE(SEWN|)NWNEEE)|EN(NESSENNNWWNEENNWN(WSWS(W(NNEWSS|)W|EE)|ENE(EEE|SS))|W))|WNNNENNWNNN(EEES(WWSNEE|)EEN(W|ESE(NE|SWW))|WSWWN(E|WSSESENESS(E|S(WNSE|)S))))|N)))))|NN))))|S)))))))|W))|SW(N|SESWS(E|WSSE(SS(S|WNWNNW(SSW(S(W(W|N)|SSSWSSWN(SENNENSWSSWN|))|N)|NENENN))|N))))|S)))|SSS(WNWESE|)SS)|S)|SSSWW(SSSSENNNESSSSSWSSE(N|SSSWNNWWSESWWW(WW|SEEESWWSSESWWNWWSSE(N|EEESEENNW(NN(W|ESE(NNWNEWSESS|)SSSSWSESWSESWWNWSSESWSESWWNNWSWWNENNWSWSSWSEEEESSENEESWSSWSEESSWNWSWWWSWNNNNWWSESWSSWSWSSSESSSWWNENWNWNENNWSWNNWNWSWNNNNESSENESSENNNNWSWNNNENEENESSWS(SSEE(NN(WSNE|)EEES(WW|ESSSEENWNEN(WNNW(S|WWWNNW(SS|NENNE(ENWNWWS(E|SSWSWNNNNE(SS|NNENNWWNENEES(W|EE(NWN(WWNWWS(E|WWNENWNWNWSSWSSSENNE(N|SSSWWWSEEESSENNE(NWNEEWWSES|)SSE(NEWS|)SWWSESSSSSWWWNENNE(SS|NWWNWWNENWN(EESSES|WSSSSEESS(ENSW|)SWNWWN(NWWNEENNNNENWNENE(NWNENWNENE(SS|ENESEE(NWNENN(E|WWNWSSS(ENEWSW|)WNNNWSSWNWWN(EE|WNNWNNWW(WWSEEESWWWWSWWNNN(WWWWWNEENESENNWNN(E(NEN(ESENES(SWWWEEEN|)E|W|N)|S)|WWSESWWS(WNWSSSSE(NN|EESENESSWSSWSEEEESENNEEEENN(WWS(WWWW(NNN|S(WNSE|)E)|E)|EESWSSENESEESSE(NESEWNWS|)SWSWWNWSWNNWN(EESENESS(NNWSWNSENESS|)|NWSSWSEESWWWWSSEESENEN(WWW|EESESES(ENNNW(S|W)|WSSSSSE(NNNN|SWWSSSWNNWSWNNWSWWWWNNEES(ENNWWWNENNNEN(WWSSSWNWSSSE(N|SWWNWSSWWNENNENE(NWNNNWNENWWNNEES(EEEESSWWSSW(NNNEEWWSSS|)SEENNEENNNEN(E(NENWESWS|)S|WWWN(E|WSWNWWNENWWNNEN(ESS(ENSW|)W|WWSSWNNNNNW(NEE(N|SSS)|SWWWNW(SS(W|SENESSESSE(NNNNWS|EESWSSE(NENSWS|)SWSWSWNNWWSSSE(NN|EESEESWSSEE(ENNW(SWEN|)N(E|NWN(WSNE|)(N|E))|SWS(E|WNWNNNWNWWWSSSENNESSE(N|SSESWSEENESSESESWWWSSWNWSSWNWWWSSESWWSSSWNWNENWWNWNNWSWNWWWNNEENENNWNNESESEENEEEENESSSWSWSES(WWWSS(SEENNWS(NESSWWEENNWS|)|WNNNNWW(SWS(EENSWW|)WW|NEEESE(SWEN|)NN(E(E|S)|W)))|ENNESE(SWEN|)NEN(WWNNE(NNW(S|WWWWS(WWNWW(SESNWN|)NENESE(S|ENENNNWSSW(WNNE(NEEES(SSSSWENNNN|)ENNWWWNENN(WWWSE(SSWNWWSWSWNWNENNNWWSS(ENSW|)SSSESEEEE(SSWSWWWNEN(WWWN(E|NNWWSSSE(SWSSEEE(EES(ENSW|)WSSSS(E|WS(WNWNWNNN(WWWSEESSSSE(N|ESSSE(NN|SWSSEEENN(WSWENE|)ESSSENE(NWES|)SE(SE(N|SWWWSWNWWSESEEEEEN(E(EEESWSESSWSSSENNENNNESSEESENENEESEESSWNWSSSSSSENNEESESENESSESSWSWSS(EEEEENESENESEEENWWNEEESSEEENWWNNESEENWNNNWWSS(ENSW|)WWS(WWNWWSESWWNNWSWNNNEENENWNENWWSSWS(WW(SESSSSE(SWW(W|N)|E)|NWWNNNNEEEENEENEEENWNWWWNWSSE(EE|SWWSWWWWNENENNESE(SWSWENEN|)NNNNENESSS(WNSE|)EEENNWNNENESENNWWWWS(E|S(SSSEWNNN|)WNNWWSESWWNNWSSWWWSEEESWSEE(NNEWSS|)SWWSSWNNNWWSESWWWNENWWWNNEENWWN(WSWWSSW(NNNWESSS|)SWNWSSESSEENN(WSNE|)EN(W|NE(NWES|)SSEESSS(EEES(W|E(SSSE(N|SS(S|WWWNEE))|N(NWWWEEES|)E))|WWSWWNENEN(ESNW|)W))|EENESE(ENWN(WWWSNEEE|)NNEESS(WNSE|)E(SWSEWNEN|)NESEEEEN(NWSWNW(WWNWN(WSWWEENE|)E|S)|ESE(N|EEEE(NWES|)SSEE(NWES|)SWSSSENNENE(NWES|)E(SSSWW(NENSWS|)SWSSWWSESWSW(NNNNW(NEE(ESWENW|)NNWW(SEWN|)N(EENSWW|)W|S|WWW)|WSEESWSWSW(SEESSS(WWN(WSWW|NES)|S|EENWNNN(W|ESSENNNW(W|NNNEE(NWNENN|SENEESSW(SSWW(NENWWN|SES(W|ESSSESENNWNEE(NWWN(EENWN(WSNE|)N|W)|EEESSSESWWSWNW(SWW(NEWS|)S(WNSE|)EEEEENEENNNWNNNENWNNEEEESSWNW(W|SSEESSWSSW(SESWS(EEENEES(EENNNNE(SSSS|NNNWNWSSWSWNW(NEENWNNNESES(ENNNE(SSSS|NNW(WWSS(WNNWWSESWS(ESSSNNNW|)WWWWN(WSNE|)NE(EESWWEENWW|)NN|E(NEWS|)S)|NNNNWNEE(SSSS|NNNNNNWSSSSS(NNNNNEWSSSSS|))))|W)|SSWSS(S|EEENN(WSWENE|)E(NENSWS|)SSS)))|W)|WW)|NNWNEE))|NNNESE(SWEN|)N))))|N)))))|NN(WN(NESNWS|)WSW(N|WWSESW(WWNENW|S))|E)))|E)))|SWSS(E(N|EE)|WW))))))|E)|E)|WNNWNNN(W(SWNWWSSSEN(ESSWWS(EEEENWNN(SSESWWEENWNN|)|WNNWNNE(NNENWNNWNEE(SS|NW(NENSWS|)WWSSWNNNWSSSSWWN(ENNSSW|)WSWWNNWSSWWNENNNNNWN(WSSWSESWWWWSWWSESSSSWSSEESSS(ENENNW(S|NEESSENNNNWSWNNENE(SESSSSSSS(WNWSWENESE|)EEEEEEENNENWWWNWWSESWS(EENESNWSWW|)WNNW(NENEEENN(WWS(WW(NEWS|)S|E)|EEESWSS(WNNSSE|)EEN(W|NEE(SWSSWSSE(N|ESWWWN)|NNWSWN)))|SS)|NN(ENNNSSSW|)WSWW(NEWS|)S(W(SESSW(SWEN|)N|N)|E)))|WWWWNWNEE(SEENWESWWN|)NNNWWWNWWWNEEENNESESS(WNSE|)ENN(ESSNNW|)NNW(WWNWNENNWWWSE(SWWSSENESESWS(WNWWNNWSSSSEE(NWES|)SWSSWSEENNEESE(EES(ENSW|)WSSWWS(EEE|WNN(N(EESWENWW|)N|WSSWWWWNEN(EESWENWW|)NNNW(SSSWWEENNN|)NNNWNWSWSWS(EENESNWSWW|)W(WSSNNE|)NNN(NEE(SWSNEN|)NNESES(W|ESE(SSSESNWNNN|)NNWNENNNWWSWW(NNE(NEENESS(WW|ENNESESWSE(SWW(SSENEWSWNN|)N|ENNNESEESWS(WNSE|)EESSSES(W|EEENWNENENE(SE(ENWESW|)SWS(WNSE|)E|NWWSWSWS(WNNNWNENWNNWNNENNWWS(E|SWSESWS(EESWENWW|)WW(S|WNWWNNNN(WSSSSS(EE|WSWWW(SEWN|)WNN(WSWNSENE|)NENEESE(SWW(WSEEWWNE|)N|NNNWWN(WW(SESEEWWNWN|)N(WSWWN(W(SSE|NE)|E)|E)|EE)))|EESSW(N|SEENNNENE(SSWSSSW(ENNNENSWSSSW|)|NWWNENWNWN(WSSESSSE(WNNNWNSESSSE|)|EENESESW(W|SES(ESS(WNSE|)EEN(EESSW(WSSEE(NWES|)SSSENEES(WSSWWSW(S|NN(NNW(N(E|W)|S)|EE))|E(EEE|N))|N)|NW(NENWW(S|N(EE|NWNNW(S|NN(W(NE|WSE)|EEEENWNNNESE(NNEENWNWNEESE(SS|NNEEEEESE(SWS(E|W(NN|S))|EN(ESNW|)WN(E|NNWSSWWWWWNENWWSW(N|WSS(EENWESWW|)WWSSS(WNSE|)E(SWEN|)(NN|E)))))|S(W|SSSSWNWSW(NWES|)S))))))|S))|W))))))))|S)))))|S)|WS(SSWENN|)EEE(ENSW|)S))|W)))|N)|E)|E)|S))|EEENESS(E|SWW(NEWS|)SES(W|E(SS|N)))))|S))|N)|N(N|E))|ESE(SWEN|)(N|E)))|N)|WW))|N)))|ESES(W|E(NNW|SW)))|E))|NW(NEWS|)W)|NN))|E(S|E))|N(N|W|E))|E)|E(EES(WWSEWNEE|)ENEE(NWES|)S(W|S)|N))|S)|S))|E))|S)|EES(WSWENE|)E))))))))|N)))))|W)|S))|ESSSEESSS(WNNWWNN(SSEESSNNWWNN|)|EE(SWEN|)NWNENNE(SSSS|NNW(WS(E|S(WNWSNESE|)S)|N))))|W)))))))|E))|ES(ENNSSW|)S)|NEN(W|N))))|S(SESENSWNWN|)W))|S)|EE)))))|E)|SWWSSENESESESENES(SW(SESWENWN|)WWNWNWWW(EEESESNWNWWW|)|EENWN(E|WWWN(W|EE)))))))|SS)))|E))|SWWSSE(SWW|ENW))|W(N|W)))|S))|NNE(S|NNESENNNE(WSSSWNSENNNE|))))|WN(WW|EENWWNN(SSEESWENWWNN|))))))|SSSWSW(N|SEENESENES(NWSWNWESENES|)))|S)))|E))|S))|N)|W))|NN(ESNW|)NNNNNNEN(E|WWSSSSW(WSSEE(NW|SWW)|NNNNWWWNNWNENWNE(NNWSNESS|)EESESWSES(WWNNNSSSEE|)ENE(NWN(NWES|)E|S))))|WWNNWWWWNENWWSWW(SESENSWNWN|)NEN(WW(SS|WWNWSW)|N))|WW(NEWS|)W))|N)|E)|EE)|ESSEESES(ENSW|)W))|NNNN))))|E))|W)|W)))|EEN(N|W))|N)))|S)))|W))|S(S|WW))|SE(S|N)))|E))))))|NENWNW))))|N)|E))|N)|E))|EE))))|WWWWSEE(WWNEEEWWWSEE|))|SSSSSSS(ENSW|)S))))))))))))|NNE(SS|EN(WWNWWSES(S|W)|E))))|SSSS))|ENN(W|N))|SS))))))|N)|E)|S)))|S)|S))|SSS))|E)|SS)|EEE))|NN)))|N)|N(NN|ESENEN))|ESEESWSEE(SWWEEN|)NN)|E))|NNWNNNWSSSWSE(E|SWWNNNENWWNN(NESE(SWEN|)N|WSSWSWWN(E|WSSEEEENESSWWW(EEENNWESSWWW|)))))|S)|SSW(WS(E|WNWWS(ESNW|)WNW(NEWS|)S)|NN))|E)))|EEN(WNEWSE|)E)|ENEENW)|E)|E))|S)$"""

In [10]:
import re
def expanded_string(line):
    final_strings = set()
    pattern_group = re.compile(r'\(([^()]+)\)')
    # Remove ^ and $ as they're just markers
    line = line.strip('^$')
    match = pattern_group.findall(line)
    if match:
        replace = f"({match[-1]})"
        replacements = match[-1].split('|')
        for replacement in replacements:
            new_line = line.replace(replace, replacement)
            final_strings.update(expanded_string(new_line))
    else:
        final_strings.add(line)
    return final_strings


In [11]:
#my_strings = expanded_string(input_text)
print(len(my_strings))

In [1]:
s1 = set([1,2,3])
s2 = set([4,5,6])
s1.update(s2)

In [2]:
s1

{1, 2, 3, 4, 5, 6}

In [35]:
neighbors = {'e': 1, 'w': 2, 'ne': 3}
kwargs = {'directions': ['e']}
neighbors = {key:value for key, value in neighbors.items() if key in kwargs['directions']}
print(neighbors)

{'e': 1}
